In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [100]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [23]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


### MODEL 1

In [201]:
# Initializing parameters
#epochs = 30
#batch_size = 16
#n_hidden = 32
#pv = 0.5

epochs = 30
batch_size = 32
n_hidden = 128
pv = 0.25

- Defining the Architecture of LSTM

In [202]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(pv))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_30 (LSTM)               (None, 128)               70656     
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dropout_29 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [203]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [233]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 56s 8ms/step - loss: 1.0168 - acc: 0.5739 - val_loss: 0.9208 - val_acc: 0.5952
Epoch 2/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.7120 - acc: 0.6473 - val_loss: 1.1691 - val_acc: 0.5341
Epoch 3/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.6310 - acc: 0.6843 - val_loss: 0.7068 - val_acc: 0.6563
Epoch 4/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.5917 - acc: 0.7053 - val_loss: 0.5508 - val_acc: 0.7394
Epoch 5/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.4028 - acc: 0.8551 - val_loss: 0.9119 - val_acc: 0.8035
Epoch 6/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.2704 - acc: 0.9064 - val_loss: 0.5868 - val_acc: 0.8015
Epoch 7/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.2240 - acc: 0.9204 - val_loss: 0.3051 - val_acc: 0.8884
Epoch 8

In [205]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      390        93        0                   0   
STANDING                 0       96       436        0                   0   
WALKING                  0        1         0      473                  10   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        0                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           12  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 470  


In [206]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [235]:
score

[0.2275053558829899, 0.9375084832032576]

- With a simple semi tuned LSTM architecture we got 93.75% accuracy and a loss of 0.22

### LSTM Model2
<br>
stacking 2 LSTM 

In [194]:


epochs1 = 30
batch_size1= 32
n_hidden1 = 128
n_hidden2 =64
pv1 = 0.2
pv2 = 0.5

- Defining the Architecture of LSTM

In [195]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model1.add(Dropout(pv1))

model1.add(LSTM(n_hidden2))
# Adding a dropout layer
model1.add(Dropout(pv2))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 128, 128)          70656     
_________________________________________________________________
dropout_27 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_28 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [196]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [236]:
# Training the model
model1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 177s 24ms/step - loss: 1.1773 - acc: 0.4807 - val_loss: 1.1739 - val_acc: 0.4605
Epoch 2/30
7352/7352 [==============================] - 172s 23ms/step - loss: 0.7775 - acc: 0.6351 - val_loss: 0.8068 - val_acc: 0.6658
Epoch 3/30
7352/7352 [==============================] - 172s 23ms/step - loss: 0.7053 - acc: 0.6725 - val_loss: 0.7839 - val_acc: 0.6471
Epoch 4/30
7352/7352 [==============================] - 172s 23ms/step - loss: 0.6439 - acc: 0.7020 - val_loss: 0.6485 - val_acc: 0.7136
Epoch 5/30
7352/7352 [==============================] - 173s 23ms/step - loss: 0.5017 - acc: 0.7956 - val_loss: 0.4165 - val_acc: 0.8544
Epoch 6/30
7352/7352 [==============================] - 172s 23ms/step - loss: 0.3517 - acc: 0.8876 - val_loss: 0.4395 - val_acc: 0.8738
Epoch 7/30
7352/7352 [==============================] - 169s 23ms/step - loss: 0.2280 - acc: 0.9214 - val_loss: 0.5353 - val_acc: 

In [198]:
# Confusion Matrix
print(confusion_matrix(Y_test, model1.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         1        0                   0   
SITTING                  0      402        86        1                   0   
STANDING                 0       76       454        1                   0   
WALKING                  0        0         0      468                  25   
WALKING_DOWNSTAIRS       0        0         0        1                 418   
WALKING_UPSTAIRS         0        2         0       15                  33   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            26  
SITTING                            2  
STANDING                           1  
WALKING                            3  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 421  


In [199]:
score1 = model1.evaluate(X_test, Y_test)

2947/2947 [==============================] - 10s 3ms/step


In [238]:
score1

[0.2804819006140821, 0.9317240922972514]

- With a simple semi tuned stacked LSTM architecture we got 93.17% accuracy and a loss of 0.28